In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re
import os

In [2]:
def get_urls_from_youtube_with_keyword(keyword):
    titles = []
    urls = []
    
    search_keyword_encode = requests.utils.quote(keyword)
    
    url = "https://www.youtube.com/results?search_query=" + search_keyword_encode
    
    driver = wd.Chrome(executable_path="C:/Users/master14/Downloads/chromedriver_win32/chromedriver.exe") 
    
    driver.get(url)
    
    # 필터 버튼 클릭
    driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

    # 필터 기준 - 업로드 날짜 - 이번 주
    upload_date = driver.find_element_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]')
    click_list_1 = upload_date.find_elements_by_id("endpoint")
    click_list_1[2].click()
    
    # 필터 창이 내려가는 속도에 맞추기 위해서 임의로 sleep한 후에 버튼 클릭
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

    # 필터 기준 - 정렬 기준 - 관련
    sorting_std = driver.find_element_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[5]')
    click_list_5 = sorting_std.find_elements_by_id("endpoint")
    click_list_5[1].click()
    time.sleep(2)

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        time.sleep(3.0)
        
        
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_page_height == last_page_height:
            break
            
        last_page_height = new_page_height
        
    html_source = driver.page_source
    
    driver.quit()
    
    soup = BeautifulSoup(html_source, 'lxml')
    
    datas = soup.select("a#video-title")

    for data in datas:
        title = data.text.replace('\n', '')
        url = "https://www.youtube.com/" + data.get('href')
        
        titles.append(title)
        urls.append(url)
        
    return titles, urls

def crawl_youtube_page_html_sources(urls):
    html_sources = []

    for i in range(500,len(urls)): #len(urls)
        driver = wd.Chrome(executable_path="C:/Users/master14/Downloads/chromedriver_win32/chromedriver.exe") 
        driver.get(urls[i])

        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(3.0)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height


        html_source = driver.page_source
        html_sources.append(html_source)
        print("OK")

        driver.quit()
    return html_sources

def get_user_IDs_and_comments(html_sources):
    my_dataframes = []
    cnt = 0
    for html in html_sources:
        soup = BeautifulSoup(html, 'lxml')
        
        youtube_title = soup.select('#container > h1 > yt-formatted-string')
        
        youtube_title_date = soup.select('#date > yt-formatted-string')
        
        youtube_user_IDs = soup.select('#author-text > span')

        youtube_comments = soup.select('#content-text')
        
        youtube_date = soup.select('#header-author > yt-formatted-string > a')
        
        
        str_youtube_title = []
        str_youtube_title_date = []
        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_date = []
        
        cnt += 1 
        print(cnt)
        
        for i in range(len(youtube_user_IDs)):
            str_youtube_title.append(str(youtube_title[0].text))
            
            str_youtube_title_date.append(str(youtube_title_date[0].text))
            
            str_tmp = str(youtube_user_IDs[i].text)
            # print(str_tmp)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('                ','')
            str_tmp = str_tmp.strip()
            str_youtube_userIDs.append(str_tmp)

            str_tmp = str(youtube_comments[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_comments.append(str_tmp)
            
            str_tmp = str(youtube_date[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_date.append(str_tmp)
            
            
        
        pd_data = {"Title" : str_youtube_title,
                   "Date" : str_youtube_title_date,
                   "ID" : str_youtube_userIDs, 
                   "Comment" : str_youtube_comments, 
                   "Comment_date" : str_youtube_date}
        

        youtube_pd = pd.DataFrame(pd_data)

        my_dataframes.append(youtube_pd)
        
    return my_dataframes

def convert_csv_from_dataframe(titles, my_dataframes):
    for i in range(len(my_dataframes)):
        title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', titles[i])
        my_dataframes[i].to_csv("data/{}.csv".format(title), encoding='utf-8-sig', index = False)        

In [3]:
titles, url = get_urls_from_youtube_with_keyword("보궐선거")

html_sources = crawl_youtube_page_html_sources(url)

my_dataframes = get_user_IDs_and_comments(html_sources)

convert_csv_from_dataframe(titles, my_dataframes)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [7]:
my_dataframes

[Empty DataFrame
 Columns: [Title, Date, ID, Comment, Comment_date]
 Index: [],
                                   Title          Date                ID  \
 0   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.     German Empire   
 1   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.           박Keyman   
 2   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.           진정코리아TV   
 3   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.               김혜수   
 4   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.                 建   
 5   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.              복음통일   
 6   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.               트럼프   
 7   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.             노르웨이숲   
 8   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.               임인기   
 9   '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.               경북댁   
 10  '보궐선거 왜하죠?' 문구 불허…선관위 '편파 논란' [포커스]  2021. 3. 31.      bong soo son   
 11  '보궐

# 파일 모두 이어붙이기

In [8]:
path ='data'
file_list = os.listdir(path)

In [9]:
total = pd.read_csv(path +  '/' + file_list[0])
for i in range(len(file_list)) :
        total =  pd.concat([total,pd.read_csv(path +  '/' + file_list[i])])

In [10]:
total

,Title,Date,ID,Comment,Comment_date
0,"4·7 보궐 마지막 여론조사...""오세훈, 큰 격차 우세"" / YTN",2021. 4. 1.,교잉,ㅋㅋㅋ 선거후 여런조사 기관 여럿 문닫는다 ㅋㅋㅋㅋ,4일 전
1,"4·7 보궐 마지막 여론조사...""오세훈, 큰 격차 우세"" / YTN",2021. 4. 1.,Hey Jin,결과보고 부끄러울텐데 ㅋㅋ,4일 전
2,"4·7 보궐 마지막 여론조사...""오세훈, 큰 격차 우세"" / YTN",2021. 4. 1.,차차,오저씨 빨아대는 댓글 너무 만타ㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏ,2일 전
3,"4·7 보궐 마지막 여론조사...""오세훈, 큰 격차 우세"" / YTN",2021. 4. 1.,황의준,뚜껑 열면 답이다 여론조사는 ㅍ ㅇ ㄷ,4일 전
4,"4·7 보궐 마지막 여론조사...""오세훈, 큰 격차 우세"" / YTN",2021. 4. 1.,신적폐청산,나이스.,4일 전
...,...,...,...,...,...
0,재·보궐선거 D-4 마지막 전략은?_강적들 379회 예고 TV CHOSUN 2104...,2021. 4. 1.,Chan Oh,박영선씨..... 우리도 도쿄에 아파트 사게 해주세요!,2일 전
1,재·보궐선거 D-4 마지막 전략은?_강적들 379회 예고 TV CHOSUN 2104...,2021. 4. 1.,김재근,"반드시 투표합시다.꼼수 정권, 내로남불의 추잡한 정권을심판합시다.",3일 전
2,재·보궐선거 D-4 마지막 전략은?_강적들 379회 예고 TV CHOSUN 2104...,2021. 4. 1.,허연경,오세훈후보님 힘내세요화이팅항상 응원하겠습니다,2일 전(수정됨)
3,재·보궐선거 D-4 마지막 전략은?_강적들 379회 예고 TV CHOSUN 2104...,2021. 4. 1.,이숙희,문재인 정권이 4년동안 한결같이 보여준 비상식적인 쇼정치에 병들었다.지긋지긋하고 신...,3일 전


In [11]:
total.to_csv('data_total/보궐선거.csv', encoding='utf-8-sig')